In [1]:
using CSV, DataFrames

In [2]:
primAuthor = "Stoleriu"
fileOwnPapers = "LS_savedrecs_mine.txt"
fileCitations = "LS_savedrecs_872.txt"

"LS_savedrecs_872.txt"

In [3]:
root = dirname(@__FILE__)
inputAIS = joinpath(root, "AI_factor_full_1997-2021.txt")
inputISI = joinpath(root, "IF_factor_full_1997-2021.txt")

"/home/lali/TITAN-ROG-sync/julia/ISI/IF_factor_full_1997-2021.txt"

In [4]:
ANini = 2017
ANfin = 2021

2021

In [5]:
AIS = CSV.read(inputAIS, DataFrame; header=true)
AIS[8157, [1, 2, 3, 14, 25, 26]]

,Journal,1997,1998,2009,2020,2021
,String31,Float64,Float64,Float64,Float64,Float64
8157,IEEE MAGN LETT,0.789,0.789,0.789,0.409,0.409


In [6]:
ISI = CSV.read(inputISI, DataFrame; header=true)
ISI[8157, [1, 2, 3, 14, 25, 26]]

,Journal,1997,1998,2009,2020,2021
,String31,Float64,Float64,Float64,Float64,Float64
8157,IEEE MAGN LETT,1.692,1.692,1.692,1.549,1.549


In [7]:
root = dirname(@__FILE__)
inputMINE = joinpath(root, fileOwnPapers)
inputCIT = joinpath(root, fileCitations)

"/home/lali/TITAN-ROG-sync/julia/ISI/LS_savedrecs_872.txt"

In [8]:
MINEraw = CSV.read(inputMINE, DataFrame; header=true)
eltype.(eachcol(MINEraw));

┌ Warning: thread = 1 warning: parsed expected 67 columns, but didn't reach end of line around data row: 1. Parsing extra columns and widening final columnset
└ @ CSV /home/lali/.julia/packages/CSV/nofYz/src/file.jl:635


In [9]:
MINE = MINEraw[:, [2, 9, 24, 30, 32, 42, 45, 46, 47, 52, 53, 54, 55]];
eltype.(eachcol(MINE));

In [10]:
MINE[1, :PY] = 2021   # ADV MAT no year!!!!
eltype.(eachcol(MINE));

In [11]:
#root = dirname("/mnt/d/Stoleriu/")
#outputMINE = joinpath(root, "MINE_test.csv")
#CSV.write(outputMINE, MINE)

Pregatim MINE cu noi coloane

In [12]:
rename!(MINE,:"J9" => :"Journal")
insertcols!(MINE, :NA => 0);
insertcols!(MINE, :NEFF => 0.0);
insertcols!(MINE, :PRIM => 0);
insertcols!(MINE, :AI => 0.0);
insertcols!(MINE, :AINEFF => 0.0);
insertcols!(MINE, :IF => 0.0);
insertcols!(MINE, :IFNA => 0.0);
insertcols!(MINE, :TIMESC => 0);
insertcols!(MINE, :CT => 0.0);
### 5 ani
insertcols!(MINE, :PRIM5 => 0);
insertcols!(MINE, :AINEFF5 => 0.0);
insertcols!(MINE, :IFNA5 => 0.0);
insertcols!(MINE, :TIMESC5 => 0);
insertcols!(MINE, :CT5 => 0.0);

Gasim numarul de autori si prim autor

In [13]:
for (row_index, lucrare) in enumerate(eachrow(MINE))
    MINE[row_index, :NA] = (1+count(";", lucrare[:AU]))

    if MINE[row_index, :NA] <= 5
        MINE[row_index, :NEFF] = MINE[row_index, :NA]
    elseif MINE[row_index, :NA] <= 15
        MINE[row_index, :NEFF] = (MINE[row_index, :NA] + 5.0) / 2.0
    elseif MINE[row_index, :NA] <= 75
        MINE[row_index, :NEFF] = (MINE[row_index, :NA] + 15.0) / 3.0
    else
        MINE[row_index, :NEFF] = (MINE[row_index, :NA] + 45.0) / 4.0
    end

    if findfirst(primAuthor, MINE[row_index, :AU])[1] == 1
        MINE[row_index, :PRIM] = 1
        if (MINE[row_index, :PY] <= ANfin) && (MINE[row_index, :PY] >= ANini)
            MINE[row_index, :PRIM5] = 1
        end
    elseif occursin(primAuthor, lucrare[:RP])
        MINE[row_index, :PRIM] = 1
        if (MINE[row_index, :PY] <= ANfin) && (MINE[row_index, :PY] >= ANini)
            MINE[row_index, :PRIM5] = 1
        end
    end
    #println(row_index, MINE[row_index, [:NA]])
    #println(row_index, MINE[row_index, [:PRIM]])
end

In [14]:
#MINE[:, [6, 7, 8, 12, 14, 15, 16, 17]]
MINE.PY = Int64.(MINE.PY)
MINE[ismissing.(MINE[!, :PY]), [1, 6, 7, 8, 12, 14, 15, 16, 17]]

,AU,Journal,PY,VL,AR,NA,NEFF,PRIM,AI
,String,String31?,Int64,Int64?,String7?,Int64,Float64,Int64,Float64


In [15]:
#root = dirname("/mnt/d/Stoleriu/")
#outputMINE = joinpath(root, "MINE_test.csv")
#CSV.write(outputMINE, MINE)

In [16]:
AISsubsect = filter(r -> any(occursin.(Vector(MINE[1, [:Journal]]), r.Journal)), AIS)
#subsect2 = filter(r -> (["ADV MATER"][1] .== r.Journal[1]), AIS)
size(AISsubsect, 1)
#["ADV MATER"][1] .== subsect[1, [:Journal]][1]

10

In [17]:
for (row_index, lucrare) in enumerate(eachrow(MINE))
    #lucrare = MINE[1, :]
    if(ismissing(lucrare[:Journal]))
        continue
    end
    for AISrow in eachrow(AIS)
        if( lucrare[:Journal] == AISrow[:Journal] )
            anString = string(lucrare[:PY])
            #println(AISrow[anString])
            MINE[row_index, :AI] = AISrow[anString]
            MINE[row_index, :AINEFF] = MINE[row_index, :AI] / MINE[row_index, :NEFF]
            if (MINE[row_index, :PY] <= ANfin) && (MINE[row_index, :PY] >= ANini)
                MINE[row_index, :AINEFF5] = MINE[row_index, :AINEFF]
            end
            
        end
    end
    for ISIrow in eachrow(ISI)
        if( lucrare[:Journal] == ISIrow[:Journal] )
            anString = string(lucrare[:PY])
            #println(ISIrow[anString])
            MINE[row_index, :IF] = ISIrow[anString]
            MINE[row_index, :IFNA] = ISIrow[anString] / MINE[row_index, :NA]
            if (MINE[row_index, :PY] <= ANfin) && (MINE[row_index, :PY] >= ANini)
                MINE[row_index, :IFNA5] = MINE[row_index, :IFNA]
            end
        end
    end
end

In [18]:
#root = dirname("/mnt/d/Stoleriu/")
#outputMINE = joinpath(root, "MINE_test.csv")
#CSV.write(outputMINE, MINE)

In [19]:
I = sum(MINE[!, :AINEFF])

14.938599034299036

In [20]:
P = sum(MINE[!, :PRIM] .* MINE[!, :AI])

10.668

In [21]:
I5 = sum(MINE[!, :AINEFF5])

1.977089826839827

In [22]:
P5 = sum(MINE[!, :PRIM5] .* MINE[!, :AI])

2.2920000000000003

############################
Citations
############################

In [23]:
MINECITEraw = CSV.read(inputCIT, DataFrame; header=true);
# autocitari
#for (row_indexCIT, lucrareCIT) in enumerate(eachrow(MINECITEraw))
#    if( occursin(primAuthor, lucrareCIT.AU) )
#        println(lucrareCIT.AU)
#    end
#end
eltype.(eachcol(MINECITEraw));

┌ Warning: thread = 1 warning: parsed expected 67 columns, but didn't reach end of line around data row: 1. Parsing extra columns and widening final columnset
└ @ CSV /home/lali/.julia/packages/CSV/nofYz/src/file.jl:635


In [24]:
# eliminare autocitari
MINECITEraw = filter(row -> !(occursin(primAuthor, row.AU)),  MINECITEraw);
for (row_indexCIT, lucrareCIT) in enumerate(eachrow(MINECITEraw))
    if( occursin(primAuthor, lucrareCIT.AU) )
        println(lucrareCIT.AU)
    end
end

In [25]:
MINECITE = MINECITEraw[:, [2, 9, 30, 42, 45, 46, 47, 52, 53, 54, 55]];
eltype.(eachcol(MINECITE));

In [26]:
#CITES = fill!(Vector{Vector{Int64}}(undef, size(MINE)[1]), [0]);
CITES = Vector{Int64}[];
for i in 1:size(MINE)[1]
    push!(CITES, [0])
end

In [27]:
CITES5 = Vector{Int64}[];
for i in 1:size(MINE)[1]
    push!(CITES5, [0])
end

In [28]:
for (row_indexMINE, lucrareMINE) in enumerate(eachrow(MINE))
    if ( ismissing(lucrareMINE[:DI]) )
        #println("Skip paper $row_indexMINE noDOI")
        continue
    end
    #println("Analyzing paper ", lucrareMINE[:DI])
    for (row_indexCIT, lucrareCIT) in enumerate(eachrow(MINECITE))
        if ( occursin(lucrareMINE[:DI], lucrareCIT[:CR]) )
            #println("Paper $row_indexMINE cited by $row_indexCIT")
            push!(CITES[row_indexMINE], row_indexCIT)
            if ( !ismissing(lucrareCIT[:PY]) )
                if (lucrareCIT[:PY] <= ANfin) && (lucrareCIT[:PY] >= ANini)
                    push!(CITES5[row_indexMINE], row_indexCIT)
                end
            end
        end
    end
    MINE[row_indexMINE, :TIMESC] = size(CITES[row_indexMINE])[1]-1
    MINE[row_indexMINE, :CT] = MINE[row_indexMINE, :TIMESC] / MINE[row_indexMINE, :NEFF]
    MINE[row_indexMINE, :TIMESC5] = size(CITES5[row_indexMINE])[1]-1
    MINE[row_indexMINE, :CT5] = MINE[row_indexMINE, :TIMESC5] / MINE[row_indexMINE, :NEFF]

end;

In [29]:
C = sum(MINE[!, :CT])

231.34955044955046

In [30]:
C5 = sum(MINE[!, :CT5])

100.30840825840826

In [31]:
root = dirname("/mnt/e/Stoleriu/Doc/CV/auto/")
outputMINE = joinpath(root, "MINE_test.csv")
CSV.write(outputMINE, MINE)

"/mnt/e/Stoleriu/Doc/CV/auto/MINE_test.csv"